In [27]:
import numpy as np
# Sigmoid activation function
def sigmoid(z):
    print("get sig")
    return 1 / (1 + np.exp(-z))

# Derivative of the sigmoid function
def sigmoid_derivative(a):
    print("sig deriv")
    return a * (1 - a)

def cross_entropy_loss(y_pred, y_true):
    print("cross ent")
    m = y_true.shape[0]
    # Clip predictions to prevent log(0)
    y_pred_clipped = np.clip(y_pred, 1e-12, 1 - 1e-12)
    # Compute cross-entropy loss
    loss = -np.sum(y_true * np.log(y_pred_clipped) + (1 - y_true) * np.log(1 - y_pred_clipped)) / m
    return loss


In [10]:
from PIL import Image, __version__ as PILLOW_VERSION
import os
from pathlib import Path

def get_resample_filter():
    """
    Determines the appropriate resampling filter based on Pillow version.
    """
    major_version = int(PILLOW_VERSION.split('.')[0])
    if hasattr(Image, 'Resampling'):
        # Pillow >= 10.0
        print("Using Pillow Resampling module.")
        return Image.Resampling.LANCZOS
    else:
        # Pillow < 10.0
        print("Using legacy resampling filters.")
        return Image.LANCZOS  # ANTIALIAS is an alias for LANCZOS in older versions

def resize_image(source_image_path, target_image_path, new_size, resample_filter):
    """
    Resizes a single image and saves it to the target path.
    
    :param source_image_path: Path to the source image.
    :param target_image_path: Path where the resized image will be saved.
    :param new_size: Tuple specifying the new size (width, height).
    :param resample_filter: Resampling filter to use.
    """
    try:
        with Image.open(source_image_path) as img:
            print(f"Processing image: {source_image_path}")
            resized_img = img.resize(new_size, resample=resample_filter)
            resized_img.save(target_image_path)
            print(f"Saved resized image to: {target_image_path}")
    except Exception as e:
        print(f"Failed to process {source_image_path}: {e}")

def process_folders(source_base_dir, target_base_dir, start=13, end=62, new_size=(80, 60)):
    """
    Iterates through folders named Sample{number} and resizes images within them.
    
    :param source_base_dir: Base directory containing source folders.
    :param target_base_dir: Base directory where resized images will be saved.
    :param start: Starting number for folder names.
    :param end: Ending number for folder names.
    :param new_size: Desired size for the resized images.
    """
    # Define supported image extensions
    supported_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff')
    
    # Get the appropriate resampling filter
    resample_filter = get_resample_filter()
    
    for i in range(3, 12 + 1):
        # Format the folder name with leading zeros (e.g., Sample013)
        folder_name = f"Sample{i:03}"
        
        # Define source and target directory paths
        source_dir = Path(source_base_dir) / folder_name
        target_dir = Path(target_base_dir) / folder_name
        
        print(f"\nProcessing Folder: {folder_name}")
        print(f"Source Directory: {source_dir}")
        print(f"Target Directory: {target_dir}")
        
        # Check if source directory exists
        if not source_dir.exists() or not source_dir.is_dir():
            print(f"Source directory does not exist or is not a directory: {source_dir}")
            continue  # Skip to the next folder
        
        # Create the target directory if it doesn't exist
        target_dir.mkdir(parents=True, exist_ok=True)
        print(f"Ensured target directory exists: {target_dir}")
        
        # Iterate over all files in the source directory
        for filename in os.listdir(source_dir):
            if filename.lower().endswith(supported_extensions):
                source_path = source_dir / filename
                target_path = target_dir / filename
                
                # Resize and save the image
                resize_image(source_path, target_path, new_size, resample_filter)
        
        print(f"All images in {folder_name} have been processed.")

    print("\nAll folders have been processed successfully.")
if __name__ == "__main__":
    # Example usage for multiple images

    source_directory = "./FinalProjectData/"    # Replace with your source directory
    target_directory = "./FinalProjectDataResized/"    # Replace with your target directory
    new_dimensions = (80, 60)                        # Desired size

    process_folders(source_directory, target_directory,13,62 ,new_dimensions)


Using Pillow Resampling module.

Processing Folder: Sample003
Source Directory: FinalProjectData\Sample003
Target Directory: FinalProjectDataResized\Sample003
Ensured target directory exists: FinalProjectDataResized\Sample003
Processing image: FinalProjectData\Sample003\img003-001.png
Saved resized image to: FinalProjectDataResized\Sample003\img003-001.png
Processing image: FinalProjectData\Sample003\img003-002.png
Saved resized image to: FinalProjectDataResized\Sample003\img003-002.png
Processing image: FinalProjectData\Sample003\img003-003.png
Saved resized image to: FinalProjectDataResized\Sample003\img003-003.png
Processing image: FinalProjectData\Sample003\img003-004.png
Saved resized image to: FinalProjectDataResized\Sample003\img003-004.png
Processing image: FinalProjectData\Sample003\img003-005.png
Saved resized image to: FinalProjectDataResized\Sample003\img003-005.png
Processing image: FinalProjectData\Sample003\img003-006.png
Saved resized image to: FinalProjectDataResized\S

In [ ]:
def load_and_split_data(data_dir, image_size=(60, 80), train_per_class=50, test_per_class=5):
    """
    Loads, processes, and splits images into training and testing datasets.

    Parameters:
    - data_dir (str): Path to the main directory containing class folders.
    - image_size (tuple): Desired image size (width, height).
    - train_per_class (int): Number of training samples per class.
    - test_per_class (int): Number of testing samples per class.

    Returns:
    - X_train (np.ndarray): Training data of shape (num_train_samples, 4800).
    - Y_train (np.ndarray): One-hot encoded training labels of shape (num_train_samples, 62).
    - X_test (np.ndarray): Testing data of shape (num_test_samples, 4800).
    - Y_test (np.ndarray): One-hot encoded testing labels of shape (num_test_samples, 62).
    """
    X_train = []
    Y_train = []
    X_test = []
    Y_test = []

    # Get a sorted list of class folder names to maintain consistent label ordering
    class_folders = sorted([folder for folder in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, folder))])
    print(f"Class folders: {class_folders}")

    # Ensure there are exactly 62 classes
    assert len(class_folders) == 62, f"Expected 62 classes, but found {len(class_folders)}."
    print(f"Number of classes: {len(class_folders)}")

    for label_index, folder in enumerate(class_folders):
        folder_path = os.path.join(data_dir, folder)
        print(f"Processing folder: {folder_path} with label index: {label_index}")

        # List all image files in the current class folder
        image_files = [file for file in os.listdir(folder_path) if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
        print(f"Image files: {image_files}")

        # Check if there are enough samples per class
        if len(image_files) < (train_per_class + test_per_class):
            raise ValueError(f"Not enough samples in {folder_path}. Required: {train_per_class + test_per_class}, Found: {len(image_files)}")
        print(f"Number of samples: {len(image_files)}")

        # Randomly shuffle the image files


        # Split into training and testing
        train_files = image_files[:train_per_class]
        test_files = image_files[train_per_class:train_per_class + test_per_class]
        print(f"Training files: {train_files}")
        print(f"Testing files: {test_files}")

        # Process training images
        for image_file in train_files:
            image_path = os.path.join(folder_path, image_file)
            print(f"Processing training image: {image_path}")

            try:
                # Open the image file
                with Image.open(image_path) as img:
                    # Convert image to grayscale ('L' mode)
                    img = img.convert('L')


                    # Convert image to NumPy array
                    img_array = np.array(img)

                    # Binarize the image: Convert to 0 and 1
                    threshold = 128
                    img_binary = (img_array > threshold).astype(np.float32)

                    # Flatten the 2D image to 1D vector of size 4800
                    img_flatten = img_binary.flatten()  # Shape: (4800,)

                    # Append to X_train
                    X_train.append(img_flatten)

                    # Create one-hot encoded label and append to Y_train
                    one_hot_label = np.zeros(62)
                    one_hot_label[label_index] = 1
                    Y_train.append(one_hot_label)

            except Exception as e:
                print(f"Error processing image {image_path}: {e}")
                continue  # Skip this image and continue with the next

        # Process testing images
        for image_file in test_files:
            image_path = os.path.join(folder_path, image_file)
            print(f"Processing testing image: {image_path}")

            try:
                # Open the image file
                with Image.open(image_path) as img:
                    # Convert image to grayscale ('L' mode)
                    img = img.convert('L')

                    # Resize the image if it's not already 80x60
                    if img.size != image_size:
                        img = img.resize(image_size)

                    # Convert image to NumPy array
                    img_array = np.array(img)

                    # Binarize the image: Convert to 0 and 1
                    threshold = 128
                    img_binary = (img_array > threshold).astype(np.float32)

                    # Flatten the 2D image to 1D vector of size 4800
                    img_flatten = img_binary.flatten()  # Shape: (4800,)

                    # Append to X_test
                    X_test.append(img_flatten)

                    # Create one-hot encoded label and append to Y_test
                    one_hot_label = np.zeros(62)
                    one_hot_label[label_index] = 1
                    Y_test.append(one_hot_label)

            except Exception as e:
                print(f"Error processing image {image_path}: {e}")
                continue  # Skip this image and continue with the next

    # Convert lists to NumPy arrays
    X_train = np.array(X_train)  # Shape: (62*50=3100, 4800)
    Y_train = np.array(Y_train)  # Shape: (3100, 62)
    X_test = np.array(X_test)    # Shape: (62*5=310, 4800)
    Y_test = np.array(Y_test)    # Shape: (310, 62)


    print(f"Total training samples: {X_train.shape[0]}")
    print(f"Shape of X_train: {X_train.shape}")
    print(f"Shape of Y_train: {Y_train.shape}")
    print(f"Total testing samples: {X_test.shape[0]}")
    print(f"Shape of X_test: {X_test.shape}")
    print(f"Shape of Y_test: {Y_test.shape}")

    return X_train, Y_train, X_test, Y_test

def preprocess_data(X):
    """
    Normalizes the input data to [0, 1].

    Parameters:
    - X (np.ndarray): Input data array.

    Returns:
    - X_normalized (np.ndarray): Normalized input data.
    """
    # Since X is already binary (0 and 1), normalization is technically redundant.
    # However, to ensure data type is float and consistent scaling:
    X_normalized = X / 1.0  # Ensures data type is float and values are 0 or 1
    return X_normalized

def one_hot_encode(labels, num_classes):
    """
    Converts integer labels to one-hot encoded vectors.

    Parameters:
    - labels (np.ndarray): Array of integer labels. Shape: (num_samples,)
    - num_classes (int): Total number of classes.

    Returns:
    - one_hot (np.ndarray): One-hot encoded labels. Shape: (num_samples, num_classes)
    """
    one_hot = np.zeros((labels.size, num_classes))
    one_hot[np.arange(labels.size), labels] = 1
    return one_hot

def main():
    # Path to the main data directory containing 62 class folders (Sample000 to Sample061)
    data_directory = './FinalProjectDataResized/'  # Replace with your actual data directory path

    # Load and split the data
    X_train, Y_train, X_test, Y_test = load_and_split_data(
        data_dir=data_directory,
        image_size=(80, 60),
        train_per_class=50,
        test_per_class=5
    )

    # Preprocess the data (normalization)
    X_train = preprocess_data(X_train)
    print("X train preproced")
    X_test = preprocess_data(X_test)
    print("X test preproced")
    return X_train, Y_train, X_test, Y_test

X_train, Y_train, X_test, Y_test = main()


In [ ]:
import numpy as np

# Activation functions and derivatives
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_derivative(z):
    return sigmoid(z) * (1 - sigmoid(z))

def relu(z):
    return np.maximum(0, z)

def relu_derivative(z):
    return (z > 0).astype(float)

def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Avoid numerical instability
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)


In [185]:
from sklearn.utils import shuffle

# Shuffle X and Y together
X_train, Y_train = shuffle(X_train, Y_train, random_state=42)
X_test, Y_test = shuffle(X_test, Y_test, random_state=42)


In [15]:
print(f"Shape of X_train: {X_train.shape}")

Shape of X_train: (3100, 4800)


In [43]:
import numpy as np

# Network architecture
input_size = 4800      # Number of input neurons (80x60 pixels flattened)
hidden_size = 100      # Number of hidden neurons
output_size = 62       # Number of output neurons (26 uppercase + 26 lowercase letters + 10 digits)

# Hyperparameters
learning_rate = 0.001  
epochs = 10
batch_size = 3100

# Initialize weights and biases
np.random.seed(42)  # For reproducibility

# Weights initialization with small random values
W1 = np.random.randn(input_size, hidden_size) * 0.01  # Shape: (4800, 100)
W2 = np.random.randn(hidden_size, output_size) * 0.01 # Shape: (100, 62)

# Biases initialization to 1
b1 = np.ones((1, hidden_size))  # Shape: (1, 100)
b2 = np.ones((1, output_size))  # Shape: (1, 62)
